In [ ]:
import torch
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import torch_directml

#一、准备数据
#1、定义超参数
Batch_size= 64
Learining_rate = 0.001
Epochs = 5
#2、定义数据转换标准
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,),(0.353,))
])
#3、下载数据
train_datasets = datasets.FashionMNIST(root='./data',train = True,download=True,transform = transform)
test_datasets = datasets.FashionMNIST(root='./data',train = False,download = True, transform = transform)
#4、使用搬运工
train_loader = torch.utils.data.DataLoader(train_datasets,batch_size=Batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_datasets,batch_size = Batch_size,shuffle =False)
#二、模型构建
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
device = torch_directml.device() if torch_directml.is_available() else "cpu"
#初始化模型
model = Net().to(device)
#三、选择裁判和教练
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=Learining_rate)
#损失函数
#优化器
#四、循环训练
def train(dataloader, model,loss_fn,optimizer):
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)
        pred = model(X)#前向传播
        loss =loss_fn(pred,y)#计算损失
        optimizer.zero_grad()#进行反向传播前清空梯度，防止叠加
        loss.backward()#反向传播
        optimizer.step()#更新Forward -> Loss -> Backward -> Step。
        if batch%100==0:
            print(f"loss:{loss.item():>7f}")
#五、测试和运行
def test(dataloader,model,loss_fn):
    size=len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred =model(X)
            test_loss+=loss_fn(pred,y).item()
            correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /=num_batches
    correct /=size
    print(f"识别准确率：{(100*correct):>0.1f}%,平均损失:{test_loss:>8f}\n")

print("训练")
for t in range(Epochs):
    print(F"===第{t+1}轮===")
    train(train_loader,model,criterion,optimizer)
    test(test_loader,model,criterion)
print("完成训练")